<a href="https://colab.research.google.com/github/Razanhus/Razan-mini-projects/blob/main/Text_to_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial, we'll use a transformer model to generate text based on input text. This is commonly referred to as text generation, and models like GPT (Generative Pretrained Transformer) and T5 are great for this task. We’ll use the popular model t5-base, which is a general-purpose text-to-text transformer that can perform tasks like summarization, translation, and text generation.

# Import the necessary libraries:

In [ ]:
from transformers import pipeline


In [ ]:

from transformers import pipline

Create a text generation pipeline:
# We’ll initialize a text-to-text generation pipeline using the t5-base model.

In [ ]:
generator = pipeline("text2text-generation", model="t5-base")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
generator = pipline(, model=)

# Provide input text:
You can provide some input text to generate a response. For this tutorial, we’ll use the T5 model in its basic form. You can experiment with tasks like answering questions or summarizing content.

In [ ]:
input_text = "Translate the following English text to French: Hugging Face is creating amazing tools for the NLP community."


# Generate the output:
Now you can use the generator() function to generate text based on the input.

In [ ]:
output = generator(input_text)
print("Generated text:", output[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated text: Hugging Face crée des outils extraordinaires pour la communauté de la LNP.


In [ ]:
output = generator(input_text)
print(, output[0])


# Fine-Tuning the T5 Model for Text-to-Text Generation

In this section, we'll demonstrate how to fine-tune the `t5-base` model for text-to-text generation tasks on a custom dataset. Fine-tuning allows the model to adapt to specific text generation tasks such as translation, summarization, or other natural language processing tasks.

We will use the Hugging Face `Trainer` API to handle the fine-tuning process.


In [ ]:
!pip install datasets

In [ ]:

# Step 1: Load a custom text-to-text dataset (e.g., a summarization dataset)
# For demonstration purposes, we’ll use the CNN/DailyMail dataset for summarization. You can replace this with your own dataset.

from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')

# Step 2: Load the pretrained model and tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Step 3: Preprocess the dataset for fine-tuning
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]  # Prepend task for T5
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Tokenize the summaries (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = dataset.map(preprocess_data, batched=True, remove_columns=["article", "highlights"])

# Step 4: Define training arguments and initialize Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
evaluation_strategy="no",  # Disable evaluation
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

# Step 5: Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
500,1.106600


TrainOutput(global_step=718, training_loss=1.0275145719310368, metrics={'train_runtime': 524.6947, 'train_samples_per_second': 5.472, 'train_steps_per_second': 1.368, 'total_flos': 1748318103797760.0, 'train_loss': 1.0275145719310368, 'epoch': 1.0})

# Now check model's performance after fine-tuning

In [ ]:
text = """
The field of natural language processing (NLP) is rapidly growing, with transformers being one of the most powerful tools.
They have revolutionized tasks like machine translation, text summarization, and question-answering. Transformers work by
using a mechanism called attention to focus on different parts of the input text. Since the introduction of the transformer
architecture, models such as BERT, GPT, and BART have emerged, each contributing to state-of-the-art performance in various
NLP tasks. Hugging Face provides easy access to these models through its library, making it accessible for both researchers
and developers to use pretrained models or fine-tune them on specific tasks.
"""

In [ ]:
#Used to save Tokenaizer

from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')

tokenizer.save_pretrained('./tokenizer_directory')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


('./tokenizer_directory/tokenizer_config.json',
 './tokenizer_directory/special_tokens_map.json',
 './tokenizer_directory/spiece.model',
 './tokenizer_directory/added_tokens.json')

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/results/checkpoint-500", tokenizer="/content/tokenizer_directory")
summarizer(text)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 145. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)


[{'summary_text': 'Natural language processing (NLP) is rapidly growing . Transformers work by using a mechanism called attention to focus on different parts of the input text . Hugging Face provides easy access to these models through its library .'}]

# Exercise Questions:






*   Experiment with different tasks, such as summarization or translation to various languages. How does the model perform when given more complex tasks (e.g., summarizing large paragraphs)?
*   Modify the script to allow the user to continuously input new tasks and generate text without having to restart the program.
*   Try out different models from Hugging Face’s text2text-generation models (e.g., t5-large, t5-small) to see how they compare in terms of performance and speed.
Explore fine-tuning the max_length and min_length parameters in the generation function to control the length of the output text.



